In [1]:
#!pip install tqdm

In [2]:
from __future__ import print_function
import os 
import pickle
import json
import requests
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import google_auth_httplib2  # This gotta be installed for build() to work
import requests_oauthlib
import xml.etree.ElementTree as ET
import os

## Parameters

In [3]:
# EVENT NAME FOR THESE PHOTOS!!
event_name = "testing 3.8 system test"


#Google Auth
SCOPES = ['https://www.googleapis.com/auth/photoslibrary.readonly']
google_secrets = 'client_secret.json'
pickle_google = "token.pickle"

# FLicker Auth
api_key = ...#HIDDEN
api_secret = ... #HIDDEN

# pipeline params
google_album_name = "full test" 
flicker_album_name = "FULL_PIPLEINE_TEST_3.8"
is_public = 0 # 1 means yes, 0 means no. 

#metadata csv
metadata_filename = "file_metadata.csv"

## Authorization Work

### Google Auth

In [4]:
#Load cred from https://console.cloud.google.com/apis/credentials?authuser=3&project=acm-pipeline-tes-1662055073652
creds = None
if(os.path.exists(pickle_google)):
    with open(pickle_google, "rb") as tokenFile:
        creds = pickle.load(tokenFile)

print(creds)
if not creds or not creds.valid:
    if (creds and creds.expired and creds.refresh_token):
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(google_secrets, SCOPES)
        creds = flow.run_local_server(port = 0)
    with open(pickle_google, "wb") as tokenFile:
        pickle.dump(creds, tokenFile)
        
service = build('photoslibrary', 'v1', credentials = creds, static_discovery=False)
service

### Flicker Auth

In [ ]:
# OAuth URLs
request_token_url = 'https://www.flickr.com/services/oauth/request_token'
access_token_url = 'https://www.flickr.com/services/oauth/access_token'
authorization_url = 'https://www.flickr.com/services/oauth/authorize'

callback_uri = "https://www.flickr.com"

oauth_session = requests_oauthlib.OAuth1Session(client_key=api_key, client_secret=api_secret, signature_method=u'HMAC-SHA1', signature_type=u'AUTH_HEADER', callback_uri=callback_uri)

# First step, fetch the request token:
request_token = oauth_session.fetch_request_token(request_token_url)

# Second step, follow this link and authorize:
print(oauth_session.authorization_url(authorization_url))

# Third step, fetch the access token:
redirect_response = input('Paste the full redirect URL here.')
print(oauth_session.parse_authorization_response(redirect_response))
access_token = oauth_session.fetch_access_token(access_token_url)
access_token

## Pipeline

### Get Google Photos

In [ ]:
google_albums = service.albums().list().execute()
albums = google_albums.get('albums', [])
for album in albums:
    if(album["title"] == google_album_name):
        break
album

In [ ]:
def get_album_photos(id, next_page_token=None):
    url = 'https://photoslibrary.googleapis.com/v1/mediaItems:search'
    if(next_page_token==None):
        payload = {
                      "albumId": id,
                    "pageSize": "5"
        }
    else:
        payload = {
                      "albumId": id,
                    "pageSize": "5",
                      "pageToken": next_page_token
        }
    
    #print(payload)
    headers = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(creds.token)
    }
    
    try:
        res = requests.request("POST", url, headers=headers, params=payload)
    except:
        print('Request error') 
    
    return(res)

photos = []
res = get_album_photos(album['id'])
photo_meta_data = res.json()
photos = photos + photo_meta_data["mediaItems"]
if ("nextPageToken" in list(photo_meta_data.keys())):
    next_page = photo_meta_data["nextPageToken"]
else:
    next_page= False

while(next_page):
    res = get_album_photos(album['id'], next_page)
    photo_meta_data = res.json()
    photos = photos + photo_meta_data["mediaItems"]
    next_page = False
    if ("nextPageToken" in list(photo_meta_data.keys())):
        next_page = photo_meta_data["nextPageToken"]
    else:
        next_page= False

    
photos

## Metadata saving

In [8]:
import pandas as pd
metadata_df = pd.DataFrame(columns=["filename", "event name", "google photos link", "raw image link", "mediaType", "Metadata", "tags", "google album", "flickr album"])
for media in photos:
    row = {
        "filename": media["filename"],
        "event name": event_name,
        "google photos link": media["productUrl"],
        "raw image link": media["baseUrl"],
        "mediaType": media['mimeType'],
        "Metadata":  media["mediaMetadata"],
        "tags": "tbd",
        "google album": album["productUrl"],
        "flickr album": "not uploaded yet"
    }
    metadata_df.loc[metadata_df.shape[0]] = row

In [9]:
metadata_df

,filename,event name,google photos link,raw image link,mediaType,Metadata,tags,google album,flickr album
0,Bike - 72566.mp4,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOUus...,video/mp4,"{'creationTime': '2021-05-01T05:21:20Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
1,Frog - 113403.mp4,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOWdN...,video/mp4,"{'creationTime': '2022-04-09T17:19:44Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
2,evan-wise-UP9ytaRlpFU-unsplash.jpg,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOWL6...,image/jpeg,"{'creationTime': '2022-09-01T21:21:07Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
3,sean-boyd-cfBSszKdpnQ-unsplash.jpg,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOVnj...,image/jpeg,"{'creationTime': '2022-09-01T21:21:09Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
4,wout-vanacker-nhBZtvfyXQE-unsplash.jpg,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOWgC...,image/jpeg,"{'creationTime': '2022-09-01T21:21:10Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
5,samsung-memory-6KNadyRGT5Q-unsplash.jpg,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOUa2...,image/jpeg,"{'creationTime': '2022-09-01T21:21:12Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
6,alaksiej-carankievic-3GptzXVSJaU-unsplash.jpg,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOUHh...,image/jpeg,"{'creationTime': '2022-09-01T21:21:13Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
7,bob-van-aubel-2ZXJjUlmzjw-unsplash.jpg,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOWyU...,image/jpeg,"{'creationTime': '2022-09-01T21:21:15Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
8,rajab-guga-32tV9Ndnzn4-unsplash.jpg,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOUGv...,image/jpeg,"{'creationTime': '2022-09-01T21:21:16Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet
9,tim-mossholder-fQR5pq4rU00-unsplash.jpg,testing 3.8 system test,https://photos.google.com/lr/album/AHQdUPwvnVA...,https://lh3.googleusercontent.com/lr/AGiIYOWQ8...,image/jpeg,"{'creationTime': '2022-09-01T21:21:18Z', 'widt...",tbd,https://photos.google.com/lr/album/AHQdUPwvnVA...,not uploaded yet


### Create Flickr Album

In [ ]:
url = 'https://api.flickr.com/services/rest/'

headers = {'Authorization': "Bearer {}".format(access_token["oauth_token"])}
params = {
    "api_key": api_key,
    "title": flicker_album_name,
    "primary_photo_id": "52328002434", #TEMP PHOTO
    "method": "flickr.photosets.create"
}
method="flickr.photosets.create"

try:
    res1 = oauth_session.post(url, params=params)
except Exception as e:
    print(e)
    print('Request error') 

mytree = ET.fromstring(res1.text)
id_of_photoset = mytree.getchildren()[0].attrib["id"]
url_of_photoset = mytree.getchildren()[0].attrib["url"]
id_of_photoset, url_of_photoset

In [ ]:
res1.text

In [12]:
is_public

0

### Add Each File To Flickr

In [ ]:
import time
import shutil
from tqdm import tqdm
for media in tqdm(photos):
    ##DOWNLOAD IMAGE
    r = requests.get(media['baseUrl'], stream = True)
    filename = media['filename']
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)

        #print('Image sucessfully Downloaded: ',filename)
    else:
        #print('Image Couldn\'t be retreived')
        continue
    
    
    ##TODO HERE:
    # ADD IN CSV STUFF FOR CATALOGING
    
    
    ##UPLOAD TO FLICKER
    url = 'https://api.flickr.com/services/upload/'

    headers = {'Authorization': "Bearer {}".format(access_token["oauth_token"])}

    params = {
        "is_public": is_public,
        "is_friend": is_public, 
        "is_family": is_public
    }

    try:
        with open(filename,'rb') as f:
            files = {'photo': f, "is_public": is_public,
        "is_friend": is_public, 
        "is_family": is_public}
            res = oauth_session.request("POST", url,  files=files)
            print(res.text)
        f.close()
    except Exception as e:
        print(e)
        #print('Request error')
        continue

    id_of_photo = None
    try:
        mytree = ET.fromstring(res.text)
        id_of_photo = mytree.getchildren()[0].text
    except Exception as e:
        #print(e)
        #print('File no exists')
        continue

    #print(res.text)
    # hold for data caps
    time.sleep(2)
    break
    
    ##ADD PHOTO TO FLICKER
    url = 'https://api.flickr.com/services/rest/'
    params = {
        "api_key": api_key,
        "photoset_id": id_of_photoset,
        "photo_id": id_of_photo,
        "method": "flickr.photosets.addPhoto"
    }

    try:
        res1 = oauth_session.request("POST", url, params=params)
        #print(res1.text)
    except Exception as e:
        print(e)
        #print('ADDING TO FLICKER ALBUM FAILED') 
        continue
   
    f.close()
    del files
    os.remove(filename)
    metadata_df.loc[metadata_df["filename"]==filename, "flickr album"] = url_of_photoset


## Clean Up

In [14]:
## Remove Temp Photo Used to Create Flicker Photo
url = 'https://api.flickr.com/services/rest/'
params = {
    "api_key": api_key,
    "photoset_id": id_of_photoset,
    "photo_id": 52328002434,
    "method": "flickr.photosets.removePhoto"
}

try:
    res1 = oauth_session.request("POST", url, params=params)
    print(res1.text)
except Exception as e:
    print(e)
    print('ADDING TO FLICKER ALBUM FAILED') 

<?xml version="1.0" encoding="utf-8" ?>
<rsp stat="ok">
</rsp>



In [15]:
metadata_df.to_csv(metadata_filename)